In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import altair as alt
from markdown import markdown
from IPython.display import Markdown
from ipywidgets.widgets import HTML, Tab
from ipywidgets import widgets
from datetime import timedelta
from matplotlib import pyplot as plt
import os.path as op

from mod import load_data, alt_theme

In [2]:
def author_url(author):
    return f"https://github.com/{author}"

In [3]:
# Parameters
fmt_date = "{:%Y-%m-%d}"

n_days = 90
start_date = fmt_date.format(pd.datetime.today() - timedelta(days=n_days))
end_date = fmt_date.format(pd.datetime.today())

renderer = "jupyterlab"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples", "nteract"]

In [4]:
# Parameters
renderer = "kaggle"
github_orgs = ["jupyterhub", "jupyter"]
start_date = "2019-01-01"
end_date = "2019-07-01"


In [5]:
comments, issues, prs = load_data('../data/')
bot_names = pd.read_csv('bot_names.csv')['names'].tolist()
comments = comments.query('author not in @bot_names').drop_duplicates()
issues = issues.query('author not in @bot_names').drop_duplicates()
prs = prs.query('author not in @bot_names').drop_duplicates()

/c/Users/chold/Dropbox/projects/jupyter-activity-snapshot/templates/mod/__init__.py:24: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  comments = pd.concat(comments)
/c/Users/chold/Dropbox/projects/jupyter-activity-snapshot/templates/mod/__init__.py:25: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  issues = pd.concat(issues)
/c/Users/chold/Dropbox/projects/jupyter-activity-snapshot/templates/mod/__init__.py:26: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future

In [6]:
# Only keep the datapoints we want
query_data = 'updatedAt > @start_date and updatedAt < @end_date and org in @github_orgs'
comments = comments.query(query_data)
issues = issues.query(query_data)
prs = prs.query(query_data)

In [7]:
alt.renderers.enable(renderer);
alt.themes.register('my_theme', alt_theme)
alt.themes.enable("my_theme")

ThemeRegistry.enable('my_theme')

In [8]:
# Information about out time window
time_delta = pd.to_datetime(end_date) - pd.to_datetime(start_date)
n_days = time_delta.days

# Information about the data we loaded
github_orgs = comments['org'].unique()

# GitHub activity

Jupyter also has lots of activity across GitHub repositories. The following sections contain
overviews of recent activity across the following GitHub organizations:

In [9]:
# Define colors we'll use for GitHub membership
author_types = ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', "NONE"]

author_palette = sns.palettes.blend_palette(["lightgrey", "lightgreen", "darkgreen"], 4)
author_colors = ["rgb({}, {}, {}, {})".format(*(ii*256)) for ii in author_palette]
author_color_dict = {key: val for key, val in zip(author_types, author_palette)}

In [10]:
orgs_md = []
for org in github_orgs:
    orgs_md.append(f'* [github.com/{org}](https://github.com/{org})')
Markdown('\n'.join(orgs_md))

* [github.com/jupyter](https://github.com/jupyter)
* [github.com/jupyterhub](https://github.com/jupyterhub)

In [11]:
Markdown(f"Showing GitHub activity from **{start_date}** to **{end_date}**")

Showing GitHub activity from **2019-01-01** to **2019-07-01**

## List of all contributors per organization

First, we'll list each contributor that has contributed to each organization in the last several days.
Contributions to open source projects are diverse, and involve much more than just contributing code and
code review. Thanks to everybody in the Jupyter communities for all that they do.

In [12]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, org in enumerate(github_orgs):
    authors_comments = comments.query('org == @org')['author']
    authors_prs = prs.query('org == @org')['author']
    unique_participants = np.unique(np.hstack([authors_comments.values, authors_prs.values]).astype(str)).tolist()
    unique_participants.sort(key=lambda a: a.lower())
    
    all_participants = [f"[{participant}](https://github.com/{participant})" for participant in unique_participants]
    participants_md = " | ".join(all_participants)
    md_html = HTML("<center>{}</center>".format(markdown(participants_md)))
    
    children = list(tabs.children)
    children.append(md_html)
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"All participants across issues and pull requests in each org in the last {n_days} days"))
display(tabs)

All participants across issues and pull requests in each org in the last 181 days

## Merged Pull requests

Here's an analysis of **merged pull requests** across each of the repositories in the Jupyter
ecosystem.

In [13]:
merged = prs.query('state == "MERGED" and closedAt > @start_date and closedAt < @end_date')

In [14]:
prs_by_repo = merged.groupby(['org', 'repo']).count()['author'].reset_index().sort_values(['org', 'author'], ascending=False)
alt.Chart(data=prs_by_repo, title=f"Merged PRs in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=prs_by_repo['repo'].values.tolist()),
    y='author',
    color='org'
)

Chart({
  data:            org                        repo  author
  47  jupyterhub         mybinder.org-deploy     121
  59  jupyterhub      zero-to-jupyterhub-k8s      83
  42  jupyterhub                  jupyterhub      70
  34  jupyterhub                   binderhub      58
  54  jupyterhub     the-littlest-jupyterhub      58
  48  jupyterhub         nativeauthenticator      34
  41  jupyterhub        jupyter-server-proxy      29
  53  jupyterhub                team-compass      22
  56  jupyterhub               traefik-proxy      20
  29  jupyterhub        alabaster-jupyterhub      13
  45  jupyterhub                 kubespawner      12
  49  jupyterhub                 nbgitpuller      12
  50  jupyterhub              oauthenticator      11
  31  jupyterhub                      binder      10
  37  jupyterhub               dockerspawner       8
  40  jupyterhub      jupyter-rsession-proxy       8
  30  jupyterhub                batchspawner       7
  36  jupyterhub     configurable-http-proxy       7
  58  jupyterhub                 yarnspawner       7
  35  jupyterhub                  chartpress       6
  52  jupyterhub              systemdspawner       6
  44  jupyterhub       kerberosauthenticator       4
  32  jupyterhub              binder-billing       3
  39  jupyterhub                  helm-chart       2
  57  jupyterhub                 wrapspawner       2
  33  jupyterhub                 binder-data       1
  38  jupyterhub       firstuseauthenticator       1
  43  jupyterhub              jupyterlab-hub       1
  46  jupyterhub            ltiauthenticator       1
  51  jupyterhub                 sudospawner       1
  55  jupyterhub            tmpauthenticator       1
  24     jupyter                 repo2docker      86
  22     jupyter                    notebook      79
  4      jupyter          enterprise_gateway      70
  7      jupyter                jupyter-book      65
  16     jupyter                   nbconvert      60
  20     jupyter                    nbgrader      50
  2      jupyter               docker-stacks      47
  10     jupyter              jupyter_client      17
  6      jupyter                     jupyter      12
  9      jupyter           jupyter.github.io      12
  18     jupyter                      nbdime      10
  21     jupyter                    nbviewer      10
  23     jupyter                   qtconsole      10
  8      jupyter              jupyter-sphinx       9
  11     jupyter                jupyter_core       9
  15     jupyter                       nb2kg       9
  3      jupyter       enhancement-proposals       7
  14     jupyter              kernel_gateway       6
  12     jupyter              jupyter_server       5
  13     jupyter                    jvm-repr       4
  27     jupyter                   telemetry       4
  28     jupyter                   terminado       3
  19     jupyter                    nbformat       2
  0      jupyter               accessibility       1
  1      jupyter  cookiecutter-docker-stacks       1
  5      jupyter                        help       1
  17     jupyter          nbconvert-examples       1
  25     jupyter    sphinxcontrib_github_alt       1
  26     jupyter                     surveys       1,
  encoding: FacetedEncoding({
    color: Color({
      field: 'org',
      type: 'nominal'
    }),
    x: X({
      field: 'repo',
      sort: ['mybinder.org-deploy', 'zero-to-jupyterhub-k8s', 'jupyterhub', 'binderhub', 'the-littlest-jupyterhub', 'nativeauthenticator', 'jupyter-server-proxy', 'team-compass', 'traefik-proxy', 'alabaster-jupyterhub', 'kubespawner', 'nbgitpuller', 'oauthenticator', 'binder', 'dockerspawner', 'jupyter-rsession-proxy', 'batchspawner', 'configurable-http-proxy', 'yarnspawner', 'chartpress', 'systemdspawner', 'kerberosauthenticator', 'binder-billing', 'helm-chart', 'wrapspawner', 'binder-data', 'firstuseauthenticator', 'jupyterlab-hub', 'ltiauthenticator', 'sudospawner', 'tmpauthenticator', 'repo2docker', 'notebook'

### A list of merged PRs by project

Below is a tabbed readout of recently-merged PRs. Check out the title to get an idea for what they
implemented, and be sure to thank the PR author for their hard work!

In [15]:
tabs = widgets.Tab(children=[])
merged_by = {}
pr_by = {}
for ii, (org, idata) in enumerate(merged.groupby('org')):
    issue_md = []
    issue_md.append(f"#### Closed PRs for org: `{org}`")
    issue_md.append("")
    for (org, repo), prs in idata.groupby(['org', 'repo']):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")
        for _, pr in prs.iterrows():
            user_name = pr['author']
            user_url = author_url(user_name)
            pr_number = pr['number']
            pr_html = pr['url']
            pr_title = pr['title']
            pr_closedby = pr['mergedBy']
            pr_closedby_url = f"https://github.com/{pr_closedby}"
            if user_name not in pr_by:
                pr_by[user_name] = 1
            else:
                pr_by[user_name] += 1
                
            if pr_closedby not in merged_by:
                merged_by[pr_closedby] = 1
            else:
                merged_by[pr_closedby] += 1
            text = f"* [(#{pr_number})]({pr_html}): _{pr_title}_ by **[@{user_name}]({user_url})** merged by **[@{pr_closedby}]({pr_closedby_url})**"
            issue_md.append(text)
    issue_md.append('')
    markdown_html = markdown('\n'.join(issue_md))
    
    children = list(tabs.children)
    children.append(HTML(markdown_html))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
tabs

### Authoring and merging stats by repository

Let's see who has been doing most of the PR authoring and merging. The PR author is generally the
person that implemented a change in the repository (code, documentation, etc). The PR merger is
the person that "pressed the green button" and got the change into the main codebase.

In [16]:
# Prep our merging DF
merged_by_repo = merged.groupby(['org', 'repo', 'author'], as_index=False).agg({'id': 'count', 'authorAssociation': 'first'}).rename(columns={'id': "authored", 'author': 'username'})
closed_by_repo = merged.groupby(['org', 'repo', 'mergedBy']).count()['id'].reset_index().rename(columns={'id': "closed", "mergedBy": "username"})

In [17]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(merged_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"PR authors for {iorg} in the last {n_days} days"
    idata = idata.groupby('username', as_index=False).agg({'authored': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('authored', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='authored',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)

alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:             username  authored authorAssociation
    48         choldgraf        54            MEMBER
    120      kevin-bates        41            MEMBER
    43           betatim        32            MEMBER
    148          parente        32            MEMBER
    112         jhamrick        30            MEMBER
    139            minrk        30            MEMBER
    127         lresende        22            MEMBER
    160        takluyver        19            MEMBER
    159         t-makaro        16      COLLABORATOR
    116    joshuazeltser        14       CONTRIBUTOR
    14             MSeal        12            MEMBER
    31         akchinSTC         8      COLLABORATOR
    25           Zsailer         8            MEMBER
    178        yuvipanda         8            MEMBER
    171         willingc         7            MEMBER
    110         jbweston         7            MEMBER
    114       joergbrech         5       CONTRIBUTOR
    63      davidanthoff         5       CONTRIBUTOR
    151          rkdarst         5       CONTRIBUTOR
    47        ccordoba12         5      COLLABORATOR
    168          vidartf         5      COLLABORATOR
    109        javabrett         4       CONTRIBUTOR
    146          nthiery         4       CONTRIBUTOR
    20     SylvainCorlay         4            MEMBER
    82            esevan         4      COLLABORATOR
    34   alexarchambault         4            MEMBER
    128  maartenbreddels         4       CONTRIBUTOR
    111          jhamman         3       CONTRIBUTOR
    154        sigurdurb         3       CONTRIBUTOR
    108        jasmainak         3       CONTRIBUTOR
    46      captainsafia         3            MEMBER
    79          emiliotl         3       CONTRIBUTOR
    62      danielmaitre         3       CONTRIBUTOR
    101         impact27         3       CONTRIBUTOR
    36          alexrudy         3       CONTRIBUTOR
    144          mwcraig         3       CONTRIBUTOR
    24        Xarthisius         3       CONTRIBUTOR
    6     GladysNalvarte         3       CONTRIBUTOR
    113         jhermann         2       CONTRIBUTOR
    175           xmnlab         2       CONTRIBUTOR
    161      thammegowda         2       CONTRIBUTOR
    68             dleen         2       CONTRIBUTOR
    74          echarles         2       CONTRIBUTOR
    77         ellisonbg         2            MEMBER
    78           emdupre         2      COLLABORATOR
    143         mtd91429         2       CONTRIBUTOR
    141           mpacer         2            MEMBER
    137           mgeier         2       CONTRIBUTOR
    166          travigd         2       CONTRIBUTOR
    98           hroncok         2       CONTRIBUTOR,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'authored',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'PR authors for jupyter in the last 181 days',
    width: 1000
  }), Chart({
    data:            username  authored authorAssociation
    26          betatim        93            MEMBER
    81            minrk        86            MEMBER
    123       yuvipanda        64            MEMBER
    33        choldgraf        64            MEMBER
    72       leportella        34       CONTRIBUTOR
    38     consideRatio        18            MEMBER
    4    GeorgianaElena        16            MEMBER
    106       sgibson91        13            MEMBER
    27  

In [18]:
charts = []
for ii, (iorg, idata) in enumerate(closed_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"Merges for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='closed',
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:         org                        repo    username  closed
    0   jupyter               accessibility   ellisonbg       1
    1   jupyter  cookiecutter-docker-stacks     parente       1
    2   jupyter               docker-stacks       minrk       1
    3   jupyter               docker-stacks     parente      46
    4   jupyter       enhancement-proposals     Zsailer       1
    ..      ...                         ...         ...     ...
    73  jupyter    sphinxcontrib_github_alt   takluyver       1
    74  jupyter                     surveys     parente       1
    75  jupyter                   telemetry  jaipreet-s       2
    76  jupyter                   telemetry   yuvipanda       2
    77  jupyter                   terminado   takluyver       3
    
    [78 rows x 4 columns],
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for jupyter in the last 181 days',
    width: 1000
  }), Chart({
    data:             org                    repo      username  closed
    78   jupyterhub    alabaster-jupyterhub       betatim       3
    79   jupyterhub    alabaster-jupyterhub     choldgraf      10
    80   jupyterhub            batchspawner    mbmilligan       7
    81   jupyterhub                  binder       betatim       7
    82   jupyterhub                  binder     choldgraf       2
    ..          ...                     ...           ...     ...
    157  jupyterhub  zero-to-jupyterhub-k8s     choldgraf       5
    158  jupyterhub  zero-to-jupyterhub-k8s  consideRatio      38
    159  jupyterhub  zero-to-jupyterhub-k8s         minrk      25
    160  jupyterhub  zero-to-jupyterhub-k8s     sgibson91       1
    161  jupyterhub  zero-to-jupyterhub-k8s     yuvipanda       4
    
    [84 rows x 4 columns],
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for jupyterhub in the last 181 days',
    width: 1000
  })]
})

## Issues

Issues are **conversations** that happen on our GitHub repositories. Here's an
analysis of issues across the Jupyter organizations.

In [19]:
created = issues.query('state == "OPEN" and createdAt > @start_date and createdAt < @end_date')
closed = issues.query('state == "CLOSED" and closedAt > @start_date and closedAt < @end_date')

In [20]:
created_counts = created.groupby(['org', 'repo']).count()['number'].reset_index()
created_counts['org/repo'] = created_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = created_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=created_counts, title=f"Issues created in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:            org                     repo  number  \
  0      jupyter            accessibility       4   
  1      jupyter            atom-notebook       1   
  2      jupyter               dashboards       1   
  3      jupyter                   design       1   
  4      jupyter            docker-stacks      10   
  ..         ...                      ...     ...   
  58  jupyterhub  the-littlest-jupyterhub      31   
  59  jupyterhub         tmpauthenticator       1   
  60  jupyterhub            traefik-proxy       1   
  61  jupyterhub              wrapspawner       3   
  62  jupyterhub   zero-to-jupyterhub-k8s      72   
  
                                org/repo  
  0                jupyter/accessibility  
  1                jupyter/atom-notebook  
  2                   jupyter/dashboards  
  3                       jupyter/design  
  4                jupyter/docker-stacks  
  ..                                 ...  
  58  jupyterhub/the-littlest-jupyterhub  
  59         jupyterhub/tmpauthenticator  
  60            jupyterhub/traefik-proxy  
  61              jupyterhub/wrapspawner  
  62   jupyterhub/zero-to-jupyterhub-k8s  
  
  [63 rows x 4 columns],
  encoding: FacetedEncoding({
    color: Color({
      field: 'org',
      type: 'nominal'
    }),
    x: X({
      field: 'repo',
      sort: Sort(['zero-to-jupyterhub-k8s', 'jupyterhub', 'binderhub', 'the-littlest-jupyterhub', 'mybinder.org-deploy', 'kubespawner', 'dockerspawner', 'jupyter-server-proxy', 'oauthenticator', 'team-compass', 'jupyter-rsession-proxy', 'jupyterlab-hub', 'ldapauthenticator', 'nativeauthenticator', 'nbgitpuller', 'jupyterhub-deploy-docker', 'configurable-http-proxy', 'alabaster-jupyterhub', 'binder', 'dummyauthenticator', 'jupyterhub-on-hadoop', 'wrapspawner', 'batchspawner', 'chartpress', 'helm-chart', 'ltiauthenticator', 'sudospawner', 'binder-billing', 'firstuseauthenticator', 'mybinder-tools', 'systemdspawner', 'tmpauthenticator', 'traefik-proxy', 'notebook', 'help', 'repo2docker', 'nbgrader', 'jupyter-book', 'jupyter', 'enterprise_gateway', 'docker-stacks', 'nbdime', 'jupyter_client', 'jupyter_server', 'nbconvert', 'enhancement-proposals', 'jupyter_console', 'accessibility', 'jupyter-sphinx', 'qtconsole', 'nbformat', 'nbviewer', 'steering', 'jupyter_kernel_test', 'atom-notebook', 'dashboards', 'design', 'jupyter-drive', 'jupyter.github.io', 'jupyter_core', 'kernel_gateway', 'kernel_gateway_demos', 'terminado']),
      type: 'nominal'
    }),
    y: Y({
      field: 'number',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Issues created in the last 181 days'
})

In [21]:
closed_counts = closed.groupby(['org', 'repo']).count()['number'].reset_index()
closed_counts['org/repo'] = closed_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = closed_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=closed_counts, title=f"Issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:            org                        repo  number  \
  0      jupyter               atom-notebook       1   
  1      jupyter  cookiecutter-docker-stacks       2   
  2      jupyter               docker-stacks      73   
  3      jupyter          enterprise_gateway      61   
  4      jupyter                        help      13   
  5      jupyter                     jupyter       9   
  6      jupyter                jupyter-book      45   
  7      jupyter               jupyter-drive       1   
  8      jupyter              jupyter-sphinx      10   
  9      jupyter           jupyter.github.io       1   
  10     jupyter              jupyter_client       5   
  11     jupyter                jupyter_core      10   
  12     jupyter              jupyter_server       9   
  13     jupyter              kernel_gateway      10   
  14     jupyter                       nb2kg       8   
  15     jupyter                   nbconvert      49   
  16     jupyter                      nbdime      11   
  17     jupyter                    nbformat       3   
  18     jupyter                    nbgrader      48   
  19     jupyter                    nbviewer       7   
  20     jupyter                    notebook     131   
  21     jupyter                   qtconsole       9   
  22     jupyter                 repo2docker      52   
  23     jupyter                   terminado       2   
  24  jupyterhub        alabaster-jupyterhub       1   
  25  jupyterhub                batchspawner       3   
  26  jupyterhub                      binder       8   
  27  jupyterhub                   binderhub      27   
  28  jupyterhub                  chartpress       2   
  29  jupyterhub     configurable-http-proxy       2   
  30  jupyterhub               dockerspawner       8   
  31  jupyterhub       firstuseauthenticator       2   
  32  jupyterhub      jupyter-rsession-proxy       2   
  33  jupyterhub        jupyter-server-proxy      19   
  34  jupyterhub                  jupyterhub      89   
  35  jupyterhub    jupyterhub-deploy-docker       1   
  36  jupyterhub  jupyterhub-deploy-teaching       1   
  37  jupyterhub        jupyterhub-on-hadoop       5   
  38  jupyterhub              jupyterlab-hub       1   
  39  jupyterhub                 kubespawner       9   
  40  jupyterhub           ldapauthenticator       4   
  41  jupyterhub         mybinder.org-deploy      17   
  42  jupyterhub         nativeauthenticator      16   
  43  jupyterhub                 nbgitpuller      15   
  44  jupyterhub              oauthenticator       3   
  45  jupyterhub              systemdspawner       2   
  46  jupyterhub                team-compass      42   
  47  jupyterhub     the-littlest-jupyterhub      58   
  48  jupyterhub            tmpauthenticator       1   
  49  jupyterhub               traefik-proxy      13   
  50  jupyterhub                 yarnspawner       4   
  51  jupyterhub      zero-to-jupyterhub-k8s      60   
  
                                   org/repo  
  0                   jupyter/atom-notebook  
  1      jupyter/cookiecutter-docker-stacks  
  2                   jupyter/docker-stacks  
  3              jupyter/enterprise_gateway  
  4                            jupyter/help  
  5                         jupyter/jupyter  
  6                    jupyter/jupyter-book  
  7                   jupyter/jupyter-drive  
  8                  jupyter/jupyter-sphinx  
  9               jupyter/jupyter.github.io  
  10                 jupyter/jupyter_client  
  11                   jupyter/jupyter_core  
  12                 jupyter/jupyter_server  
  13                 jupyter/kernel_gateway  
  14                          jupyter/nb2kg  
  15                      jupyter/nbconvert  
  16                         jupyter/nbdime  
  17                       jupyter/nbformat  
  18                       jupyter/nbgrader  
  19                       jupyter/nbviewer  
  20                       jupyter/notebook  
  2

In [22]:
created_closed = pd.merge(created_counts.rename(columns={'number': 'created'}).drop(columns='org/repo'),
                          closed_counts.rename(columns={'number': 'closed'}).drop(columns='org/repo'),
                          on=['org', 'repo'], how='outer')

created_closed = pd.melt(created_closed, id_vars=['org', 'repo'], var_name="kind", value_name="count").replace(np.nan, 0)

In [23]:
charts = []
for org in github_orgs:
    # Pick the top 10 repositories
    this_issues = created_closed.query('org == @org')
    top_repos = this_issues.groupby(['repo']).sum().sort_values(by='count', ascending=False).head(10).index
    ch = alt.Chart(this_issues.query('repo in @top_repos'), width=120).mark_bar().encode(
        x=alt.X("kind", axis=alt.Axis(labelFontSize=15, title="")), 
        y=alt.Y('count', axis=alt.Axis(titleFontSize=15, labelFontSize=12)),
        color='kind',
        column=alt.Column("repo", header=alt.Header(title=f"Issue activity, last {n_days} days for {org}", titleFontSize=15, labelFontSize=12))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:         org                repo     kind  count
    4   jupyter       docker-stacks  created   10.0
    6   jupyter  enterprise_gateway  created   16.0
    7   jupyter                help  created   59.0
    8   jupyter             jupyter  created   19.0
    9   jupyter        jupyter-book  created   22.0
    20  jupyter           nbconvert  created    8.0
    21  jupyter              nbdime  created   10.0
    23  jupyter            nbgrader  created   26.0
    25  jupyter            notebook  created  211.0
    27  jupyter         repo2docker  created   37.0
    71  jupyter       docker-stacks   closed   73.0
    73  jupyter  enterprise_gateway   closed   61.0
    74  jupyter                help   closed   13.0
    75  jupyter             jupyter   closed    9.0
    76  jupyter        jupyter-book   closed   45.0
    87  jupyter           nbconvert   closed   49.0
    88  jupyter              nbdime   closed   11.0
    90  jupyter            nbgrader   closed   48.0
    92  jupyter            notebook   closed  131.0
    94  jupyter         repo2docker   closed   52.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 181 days for jupyter',
          titleFontSize: 15
        }),
        type: 'nominal'
      }),
      x: X({
        axis: Axis({
          labelFontSize: 15,
          title: ''
        }),
        field: 'kind',
        type: 'nominal'
      }),
      y: Y({
        axis: Axis({
          labelFontSize: 12,
          titleFontSize: 15
        }),
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    width: 120
  }), Chart({
    data:             org                     repo     kind  count
    34   jupyterhub                binderhub  created   36.0
    37   jupyterhub            dockerspawner  created   18.0
    42   jupyterhub     jupyter-server-proxy  created   17.0
    43   jupyterhub               jupyterhub  created   47.0
    47   jupyterhub              kubespawner  created   20.0
    51   jupyterhub      mybinder.org-deploy  created   24.0
    52   jupyterhub      nativeauthenticator  created    8.0
    57   jupyterhub             team-compass  created   14.0
    58   jupyterhub  the-littlest-jupyterhub  created   31.0
    62   jupyterhub   zero-to-jupyterhub-k8s  created   72.0
    101  jupyterhub                binderhub   closed   27.0
    104  jupyterhub            dockerspawner   closed    8.0
    109  jupyterhub     jupyter-server-proxy   closed   19.0
    110  jupyterhub               jupyterhub   closed   89.0
    114  jupyterhub              kubespawner   closed    9.0
    118  jupyterhub      mybinder.org-deploy   closed   17.0
    119  jupyterhub      nativeauthenticator   closed   16.0
    124  jupyterhub             team-compass   closed   42.0
    125  jupyterhub  the-littlest-jupyterhub   closed   58.0
    129  jupyterhub   zero-to-jupyterhub-k8s   closed   60.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 181 days for jupyterhub',
          titleFontSize: 15
        }),
        type: 'nominal'
      }),
      x: X({
        axis: Axis({
          labelFontSize: 15,
          title: ''
        }),
        field: 'kind',
        type: 'nominal'
      }),
      y: Y({
        axis: Axis({
          labelFontSize: 12,
          titleFontSize: 15
        }),
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    width: 120
  })]
})

In [24]:
# Set to datetime
for kind in ['createdAt', 'closedAt']:
    closed.loc[:, kind] = pd.to_datetime(closed[kind])
    
closed.loc[:, 'time_open'] = closed['closedAt'] - closed['createdAt']
closed.loc[:, 'time_open'] = closed['time_open'].dt.total_seconds()

/home/choldgraf/anaconda/envs/dev/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/home/choldgraf/anaconda/envs/dev/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


In [25]:
time_open = closed.groupby(['org', 'repo']).agg({'time_open': 'median'}).reset_index()
time_open['time_open'] = time_open['time_open'] / (60 * 60 * 24)
time_open['org/repo'] = time_open.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = time_open.sort_values(['org', 'time_open'], ascending=False)['repo'].values
alt.Chart(data=time_open, title=f"Time to close for issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y=alt.Y('time_open', title="Median Days Open"),
    color='org',
)

Chart({
  data:            org                        repo    time_open  \
  0      jupyter               atom-notebook    73.856030   
  1      jupyter  cookiecutter-docker-stacks    36.618924   
  2      jupyter               docker-stacks   153.428380   
  3      jupyter          enterprise_gateway     6.151725   
  4      jupyter                        help     0.094005   
  5      jupyter                     jupyter     1.284792   
  6      jupyter                jupyter-book     8.177106   
  7      jupyter               jupyter-drive  1181.490556   
  8      jupyter              jupyter-sphinx    16.156800   
  9      jupyter           jupyter.github.io     0.077083   
  10     jupyter              jupyter_client   141.419028   
  11     jupyter                jupyter_core   324.044091   
  12     jupyter              jupyter_server   164.074618   
  13     jupyter              kernel_gateway   368.843293   
  14     jupyter                       nb2kg     5.844606   
  15     jupyter                   nbconvert    72.274988   
  16     jupyter                      nbdime     5.252766   
  17     jupyter                    nbformat     2.239954   
  18     jupyter                    nbgrader   155.606985   
  19     jupyter                    nbviewer     6.889421   
  20     jupyter                    notebook    14.949028   
  21     jupyter                   qtconsole     2.596262   
  22     jupyter                 repo2docker    19.760573   
  23     jupyter                   terminado     7.405671   
  24  jupyterhub        alabaster-jupyterhub   223.006389   
  25  jupyterhub                batchspawner    87.364086   
  26  jupyterhub                      binder     0.895885   
  27  jupyterhub                   binderhub     6.632280   
  28  jupyterhub                  chartpress   116.557072   
  29  jupyterhub     configurable-http-proxy   507.192587   
  30  jupyterhub               dockerspawner    46.755938   
  31  jupyterhub       firstuseauthenticator     3.203177   
  32  jupyterhub      jupyter-rsession-proxy     6.126684   
  33  jupyterhub        jupyter-server-proxy     1.018125   
  34  jupyterhub                  jupyterhub     4.523299   
  35  jupyterhub    jupyterhub-deploy-docker     3.008692   
  36  jupyterhub  jupyterhub-deploy-teaching   589.675150   
  37  jupyterhub        jupyterhub-on-hadoop     5.924155   
  38  jupyterhub              jupyterlab-hub   312.035937   
  39  jupyterhub                 kubespawner    36.807315   
  40  jupyterhub           ldapauthenticator     0.754416   
  41  jupyterhub         mybinder.org-deploy     1.212639   
  42  jupyterhub         nativeauthenticator     8.624913   
  43  jupyterhub                 nbgitpuller    21.975602   
  44  jupyterhub              oauthenticator    47.958796   
  45  jupyterhub              systemdspawner    20.571036   
  46  jupyterhub                team-compass    29.973241   
  47  jupyterhub     the-littlest-jupyterhub    73.635833   
  48  jupyterhub            tmpauthenticator    50.088738   
  49  jupyterhub               traefik-proxy    30.717477   
  50  jupyterhub                 yarnspawner     1.191123   
  51  jupyterhub      zero-to-jupyterhub-k8s     3.194711   
  
                                   org/repo  
  0                   jupyter/atom-notebook  
  1      jupyter/cookiecutter-docker-stacks  
  2                   jupyter/docker-stacks  
  3              jupyter/enterprise_gateway  
  4                            jupyter/help  
  5                         jupyter/jupyter  
  6                    jupyter/jupyter-book  
  7                   jupyter/jupyter-drive  
  8                  jupyter/jupyter-sphinx  
  9               jupyter/jupyter.github.io  
  10                 jupyter/jupyter_client  
  11                   jupyter/jupyter_core  
  12                 jupyter/jupyter_server  
  13                 jupyter/kernel_gateway  
  14                          jupyter/nb2kg  
  15          

### A list of recent issues

Below is a list of issues with recent activity in each repository. If they seem of interest
to you, click on their links and jump in to participate!

In [26]:
# Add comment count data to issues and PRs
comment_counts = (
    comments
    .query("createdAt > @start_date and createdAt < @end_date")
    .groupby(['org', 'repo', 'issue_id'])
    .count().iloc[:, 0].to_frame()
)
comment_counts.columns = ['n_comments']
comment_counts = comment_counts.reset_index()

In [27]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, (org, idata) in enumerate(comment_counts.groupby('org')):
    issue_md = []
    issue_md.append(f"#### {org}")
    issue_md.append("")
    for repo, i_issues in idata.groupby('repo'):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")

        top_issues = i_issues.sort_values('n_comments', ascending=False).head(n_plot)
        top_issue_list = pd.merge(issues, top_issues, left_on=['org', 'repo', 'number'], right_on=['org', 'repo', 'issue_id'])
        for _, issue in top_issue_list.sort_values('n_comments', ascending=False).head(n_plot).iterrows():
            user_name = issue['author']
            user_url = author_url(user_name)
            issue_number = issue['number']
            issue_html = issue['url']
            issue_title = issue['title']

            text = f"* [(#{issue_number})]({issue_html}): _{issue_title}_ by **[@{user_name}]({user_url})**"
            issue_md.append(text)

    issue_md.append('')
    md_html = HTML(markdown('\n'.join(issue_md)))
    
    children = list(tabs.children)
    children.append(HTML(markdown('\n'.join(issue_md))))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"Here are the top {n_plot} active issues in each repository in the last {n_days} days"))
display(tabs)

Here are the top 5 active issues in each repository in the last 181 days

## Commenters across repositories

These are commenters across all issues and pull requests in the last several days.
These are colored by the commenter's association with the organization. For information
about what these associations mean, [see this StackOverflow post](https://stackoverflow.com/a/28866914/1927102).

In [28]:
commentors = (
    comments
    .query("createdAt > @start_date and createdAt < @end_date")
    .groupby(['org', 'repo', 'author', 'authorAssociation'])
    .count().rename(columns={'id': 'count'})['count']
    .reset_index()
    .sort_values(['org', 'count'], ascending=False)
)

In [29]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(commentors.groupby(['org'])):
    title = f"Top {n_plot} commentors for {iorg} in the last {n_days} days"
    idata = idata.groupby('author', as_index=False).agg({'count': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('count', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='author',
        y='count',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:                author  count authorAssociation
    513         choldgraf    473            MEMBER
    887       kevin-bates    406            MEMBER
    453           betatim    359            MEMBER
    1131          parente    226            MEMBER
    1365        takluyver    210            MEMBER
    818          jhamrick    187            MEMBER
    200             MSeal    186            MEMBER
    1490        yuvipanda    146            MEMBER
    1050            minrk    122            MEMBER
    1361         t-makaro     90      COLLABORATOR
    339           Zsailer     78            MEMBER
    1427          vidartf     68      COLLABORATOR
    794        jasongrout     66            MEMBER
    642           emdupre     65      COLLABORATOR
    967          lresende     63            MEMBER
    295     SylvainCorlay     63            MEMBER
    370          akhmerov     62            MEMBER
    529      consideRatio     58      COLLABORATOR
    569      davidanthoff     57       CONTRIBUTOR
    792         jasmainak     56       CONTRIBUTOR
    651            esevan     44      COLLABORATOR
    1236         rolweber     41      COLLABORATOR
    1461         willingc     41            MEMBER
    1176      psychemedia     40              NONE
    1221           rgbkrk     39            MEMBER
    1003  mathieuboudreau     38       CONTRIBUTOR
    981   maartenbreddels     35       CONTRIBUTOR
    697           gnestor     34            MEMBER
    466         blink1073     33            MEMBER
    1229          rkdarst     30       CONTRIBUTOR
    496        ccordoba12     29      COLLABORATOR
    71            Carreau     29            MEMBER
    1463           witten     29       CONTRIBUTOR
    624          echarles     28       CONTRIBUTOR
    1028           mgeier     28       CONTRIBUTOR
    6            1kastner     25              NONE
    1067           mpacer     24            MEMBER
    695          glorysdj     22              NONE
    610           dsblank     22            MEMBER
    639         ellisonbg     21            MEMBER
    267              Ruv7     21            MEMBER
    839        joergbrech     20       CONTRIBUTOR
    192      Litchilitchy     20              NONE
    817           jhamman     18       CONTRIBUTOR
    1115          nthiery     18       CONTRIBUTOR
    420            arnavs     17              NONE
    1477           xmnlab     17       CONTRIBUTOR
    852     joshuazeltser     16       CONTRIBUTOR
    776        jaipreet-s     15      COLLABORATOR
    820           jhornig     15              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'author',
        type: 'nominal'
      }),
      y: Y({
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Top 50 commentors for jupyter in the last 181 days',
    width: 1000
  }), Chart({
    data:                   author  count authorAssociation
    150              betatim    552            MEMBER
    623            yuvipanda    479            MEMBER
    184            choldgraf    411            MEMBER
    194         consideRatio    386            MEMBER
    445                minrk    364            MEMBER
    413               manics    100       CONTRIBUTOR
    97               Zsailer     67            MEMBER
    397           leportella     58      COLLABORATOR
    608             willingc     57            MEMBER
    549            sgibson91     47 

## First responders

First responders are the first people to respond to a new issue in one of the repositories.
The following plots show first responders for recently-created issues.

In [30]:
first_comments = []
for (org, repo, issue_id), i_comments in comments.groupby(['org', 'repo', 'issue_id']):
    ix_min = pd.to_datetime(i_comments['createdAt']).idxmin()
    first_comment = i_comments.loc[ix_min]
    if isinstance(first_comment, pd.DataFrame):
        first_comment = first_comment.iloc[0]
    first_comments.append(first_comment)
first_comments = pd.concat(first_comments, axis=1).T

In [31]:
first_responder_counts = first_comments.groupby(['org', 'author', 'authorAssociation'], as_index=False).\
    count().rename(columns={'id': 'n_first_responses'}).sort_values(['org', 'n_first_responses'], ascending=False)


In [32]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(first_responder_counts.groupby(['org'])):
    title = f"Top {n_plot} first responders for {iorg} in the last {n_days} days"
    idata = idata.groupby('author', as_index=False).agg({'n_first_responses': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('n_first_responses', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='author',
        y='n_first_responses',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:               author  n_first_responses authorAssociation
    238        choldgraf                119            MEMBER
    626        takluyver                115            MEMBER
    524          parente                107            MEMBER
    412      kevin-bates                100            MEMBER
    209          betatim                 95            MEMBER
    383         jhamrick                 66            MEMBER
    87             MSeal                 59            MEMBER
    623         t-makaro                 40      COLLABORATOR
    489            minrk                 36            MEMBER
    689        yuvipanda                 35            MEMBER
    659          vidartf                 28      COLLABORATOR
    162          Zsailer                 20            MEMBER
    173         akhmerov                 20            MEMBER
    448         lresende                 16            MEMBER
    574         rolweber                 15      COLLABORATOR
    373       jasongrout                 14            MEMBER
    230       ccordoba12                 13      COLLABORATOR
    372        jasmainak                 11       CONTRIBUTOR
    480           mgeier                 10       CONTRIBUTOR
    563           rgbkrk                  9            MEMBER
    137    SylvainCorlay                  9            MEMBER
    496           mpacer                  8            MEMBER
    299          emdupre                  8      COLLABORATOR
    397    joshuazeltser                  7       CONTRIBUTOR
    267     davidanthoff                  7       CONTRIBUTOR
    675         willingc                  6            MEMBER
    546      psychemedia                  6              NONE
    517          nthiery                  6       CONTRIBUTOR
    651         ttimbers                  5              NONE
    63   IamViditAgarwal                  5              NONE
    376        javabrett                  5       CONTRIBUTOR
    172        akchinSTC                  5      COLLABORATOR
    534        perllaghu                  5            MEMBER
    377         jbweston                  4            MEMBER
    384         jhermann                  4       CONTRIBUTOR
    504           mwouts                  4              NONE
    341          hroncok                  4       CONTRIBUTOR
    391       joergbrech                  4       CONTRIBUTOR
    304           esevan                  4      COLLABORATOR
    289         echarles                  4       CONTRIBUTOR
    286          dsblank                  4            MEMBER
    575          romainx                  4       CONTRIBUTOR
    672        westurner                  4       CONTRIBUTOR
    154       Xarthisius                  4       CONTRIBUTOR
    325          gnestor                  3            MEMBER
    364        jakirkham                  3            MEMBER
    302         ericdill                  3       CONTRIBUTOR
    300         emiliotl                  3       CONTRIBUTOR
    216        blink1073                  3            MEMBER
    192           arnavs                  3              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'author',
        type: 'nominal'
      }),
      y: Y({
        field: 'n_first_responses',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Top 50 first responders for jupyter in the last 181 days',
    width: 1000
  }), Chart({
   

In [33]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>